In [ ]:
-- 1
select name, record
from sports
order by name;

In [ ]:
-- 2
select distinct s.name
from results r
join sports s
on s.id = r.sportid;

In [ ]:
-- 3
select count(distinct peopleid)
from results;

In [ ]:
-- 4
select p.id, p.name 
from (
    select r.peopleid as id
    from results r
    group by r.peopleid
    having count(r.peopleid) >= 20
) as r 
join people p 
on p.id = r.id;

In [ ]:
-- 5
select distinct p.id, p.name, g.description
from sports s
join results r 
on s.id = r.sportid
and s.record = r.result
join people p 
on r.peopleid = p.id 
join gender g 
on p.gender = g.gender;

In [ ]:
-- 6
select s.name, count(distinct r.peopleid) as numathletes
from sports s 
join results r 
on s.record = r.result 
and s.id = r.sportid 
group by s.name;

In [ ]:
-- 7
select p.id, p.name, r.best, r.difference 
from (
    select 
        r.peopleid as id, 
        max(r.result) as best, 
        to_char(s.record-max(r.result), '0D99') as difference
    from results r 
    join sports s
    on s.name like 'Triple Jump'
    and s.id = r.sportid
    group by r.peopleid, s.record
    having count(r.peopleid) >= 20
) as r
join people p 
on r.id = p.id;

In [ ]:
-- 8
select distinct p.id, p.name, g.description 
from (
    select c.id from competitions c
    where c.place like 'Hvide Sande'
    and extract(year from c.held) = 2009
) as c
join results r 
on c.id = r.competitionid
join people p 
on r.peopleid = p.id
join gender g 
on g.gender = p.gender;

In [ ]:
-- 9
select p.name, g.description
from people p
join gender g 
on g.gender = p.gender
and p.name like '% J%sen'; 

In [ ]:
-- 10
select 
    p.name, 
    s.name, 
    case when r.result is null then to_char(0,'999%')
    else to_char(100*(r.result/s.record),'999%') end as percentage
from sports s 
join results r 
on s.id = r.sportid
join people p 
on r.peopleid = p.id;

In [ ]:
-- 11
select count(distinct r.peopleid)
from results r
where result is null;

In [ ]:
-- 12
select s.id, s.name, to_char(max(r.result), '999D99') as maxres 
from sports s 
join results r 
on s.id = r.sportid
group by s.id, s.name
order by s.id;

In [ ]:
-- 13
select p.id, p.name, r.records
from (
    select r.peopleid as id, count(s.id) as records
    from results r
    join sports s 
    on r.result = s.record
    and s.id = r.sportid
    group by r.peopleid
    having count(distinct s.id) >= 2
) as r
join people p
on r.id = p.id;